# Author

1. Daniel Najoan **(First Author)**
2. Ghazali Akmal Rabbani **(Second Author)**
3. Muhammad Dhiaz Rafilianza **(Third Author)**

# Import Library

In [ ]:
import pandas as pd
import numpy as np

# Read Data

In [ ]:
data = pd.read_csv("players_20.csv")

# Data Cleaning

In [ ]:
data.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,...,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,NaN,NaN,NaN,...,48,40,94,94,75,96,33,37,26,6,11,15,14,8,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2018-07-10,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,NaN,NaN,NaN,...,63,29,95,82,85,95,28,32,24,7,11,15,14,11,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,2017-08-03,2022.0,LW,10.0,91.0,85.0,87.0,95.0,32.0,58.0,NaN,NaN,NaN,...,51,36,87,90,90,94,27,26,29,9,9,15,15,11,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,90+3,90+3,90+3,89+3,82+3,82+3,82+3,89+3,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,GK,13.0,NaN,2014-07-16,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,92.0,78.0,...,34,19,11,65,11,68,27,12,18,87,92,78,90,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",LW,7.0,NaN,2019-07-01,2024.0,LF,10.0,91.0,83.0,86.0,94.0,35.0,66.0,NaN,NaN,NaN,...,54,41,87,89,88,91,34,27,22,11,12,6,8,8,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,83+3,83+3,83+3,89+3,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


**In this FIFA20 Project, we want to cluster the players based on their team position in the club to their traits and skill**

In [ ]:
data["team_position"].count()

18038

In [ ]:
data["team_position"].value_counts()

SUB    7820
RES    2958
GK      662
LCB     660
RCB     660
RB      560
LB      560
ST      458
LCM     411
RCM     411
RM      399
LM      398
CAM     311
RDM     244
LDM     242
LS      195
RS      195
CDM     181
LW      162
RW      161
CB      100
CM       76
LWB      58
RWB      58
LAM      23
RAM      23
RF       19
LF       19
CF       14
Name: team_position, dtype: int64

# Separate SUB Position FIFA Data based on Team Position

## Non Substitution and Drop Reserve Player Data

**In this step, we want to split and get the data without SUB, RES, and Null data value in team_position column**

In [ ]:
nonSub_data = data.loc[(data['team_position'] != "SUB") & (data['team_position'] != "RES") & (data['team_position'].notna())]

**Re-check the data to make sure there is not data with SUB, RES, and Null data value**

In [ ]:
nonSub_data['team_position'].isna().sum()

0

## Null Team Position Player Data &amp; Substitution Player Data

Filter the data into two new variable to get value of SUB and Null data. In this dataframe, we would like to do data cleaning with **replace SUB and Null value data with first best position of player_position column**

In [ ]:
null_data = data[data["team_position"].isna()]
sub_data = data.loc[data['team_position'] == "SUB"]

### Count the data to make sure no more missing value

In [ ]:
#To count how many Null data value
index = null_data.index
number_of_rows = len(index)
print("Count of Nan Data: ", number_of_rows)

#To count how many SUB data value
print("Count of Sub Data: ", sub_data['team_position'].count())

Count of Nan Data:  240
Count of Sub Data:  7820


Split player positions column so we can replace Team Position Column Data for:
- Null Player Data
- Substitution Player Data

In [ ]:
temp_null_position = null_data["player_positions"].str.split(",",expand = True)
temp_sub_position = sub_data["player_positions"].str.split(",",expand = True) 

In [ ]:
temp_null_position

,0,1,2
327,CB,None,None
328,CAM,RM,None
407,CB,CDM,None
408,CAM,LM,LW
409,ST,CF,None
...,...,...,...
16353,LB,None,None
16354,CM,None,None
16356,GK,None,None
16600,GK,None,None


In [ ]:
temp_sub_position

,0,1,2
48,CF,ST,None
53,GK,None,None
55,LW,None,None
59,CB,None,None
62,CAM,CM,LW
...,...,...,...
18272,GK,None,None
18274,CB,None,None
18275,CM,None,None
18276,CM,None,None


### Delete Unused Column Data

Delete column 1 and 2 in temp_sub_position and temp_null_position, because we only need the first column

In [ ]:
del temp_sub_position[1]
del temp_sub_position[2]
del temp_null_position[1]
del temp_null_position[2]

### Rename Temporary Column

In [ ]:
temp_sub_position.rename(columns={0:'player_positions'}, inplace=True)
temp_null_position.rename(columns={0:'player_positions'}, inplace=True)

In [ ]:
temp_null_position

,player_positions
327,CB
328,CAM
407,CB
408,CAM
409,ST
...,...
16353,LB
16354,CM
16356,GK
16600,GK


In [ ]:
temp_sub_position

,player_positions
48,CF
53,GK
55,LW
59,CB
62,CAM
...,...
18272,GK
18274,CB
18275,CM
18276,CM


Re-check the count of those two dataframe in case there is Null data value

In [ ]:
print("Substitution Player Data: ",temp_sub_position.count())
print("Null Player Data: ",temp_null_position.count())

Substitution Player Data:  player_positions    7820
dtype: int64
Null Player Data:  player_positions    240
dtype: int64


### Replace Sub_data &amp; Null_data 

In [ ]:
sub_data.team_position[sub_data.team_position == "SUB"] = temp_sub_position.player_positions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [ ]:
null_data.team_position[null_data["team_position"].isna()] = temp_null_position.player_positions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1021: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


### Count again to make sure no missing value

In [ ]:
sub_data["team_position"].count()

7820

In [ ]:
sub_data["team_position"].isna().sum()

0

In [ ]:
null_data["team_position"].isna().sum()

0

In [ ]:
null_data["team_position"].count()

240

In [ ]:
null_data.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,...,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
327,245309,https://sofifa.com/player/245309/egidio-maestr...,E. Schetino,Egidio Maestre Schetino,27,1992-02-29,185,80,Uruguay,Uruguay,82,82,0,0,CB,Right,1,3,2,Medium/Medium,Normal,No,NaN,NaN,CB,NaN,NaN,NaN,NaN,RCB,4.0,80.0,43.0,56.0,65.0,83.0,76.0,NaN,NaN,NaN,...,86,82,38,42,56,71,84,85,76,58,40,64,56,43,59+2,59+2,59+2,59+2,59+2,59+2,59+2,59+2,60+2,60+2,60+2,61+2,64+2,64+2,64+2,61+2,73+2,76+2,76+2,76+2,73+2,76+2,80+2,80+2,80+2,76+2
328,245313,https://sofifa.com/player/245313/jose-maria-si...,J. Sildero,José María Sildero,27,1992-02-29,177,75,Uruguay,Uruguay,82,82,0,0,"CAM, RM",Right,1,4,4,Medium/Low,Normal,No,NaN,NaN,CAM,NaN,NaN,NaN,NaN,RM,17.0,81.0,77.0,81.0,82.0,40.0,71.0,NaN,NaN,NaN,...,62,40,80,83,77,81,40,35,33,12,12,13,15,13,77+2,77+2,77+2,80+2,80+2,80+2,80+2,80+2,81+2,81+2,81+2,80+2,77+2,77+2,77+2,80+2,63+2,62+2,62+2,62+2,63+2,59+2,52+2,52+2,52+2,59+2
407,245299,https://sofifa.com/player/245299/jaime-nicolas...,J. Frendado,Jaime Nicolás Frendado,35,1984-02-29,181,82,Uruguay,Uruguay,81,81,0,0,"CB, CDM",Right,1,3,2,High/Medium,Normal,No,NaN,NaN,CB,NaN,NaN,NaN,NaN,LCB,5.0,71.0,61.0,57.0,62.0,82.0,75.0,NaN,NaN,NaN,...,86,82,58,50,62,70,83,82,82,58,62,54,71,48,66+2,66+2,66+2,60+2,63+2,63+2,63+2,60+2,61+2,61+2,61+2,60+2,63+2,63+2,63+2,60+2,69+2,72+2,72+2,72+2,69+2,72+2,80+2,80+2,80+2,72+2
408,245304,https://sofifa.com/player/245304/saul-marcelo-...,S. Ardero,Saúl Marcelo Ardero,31,1988-02-29,171,63,Uruguay,Uruguay,81,81,0,0,"CAM, LM, LW",Left,1,5,4,Medium/Medium,Lean,No,NaN,#Acrobat,CAM,NaN,NaN,NaN,NaN,LM,16.0,82.0,72.0,77.0,83.0,26.0,51.0,NaN,NaN,NaN,...,41,31,82,80,74,77,31,13,23,9,7,8,14,10,72+2,72+2,72+2,79+2,78+2,78+2,78+2,79+2,80+2,80+2,80+2,79+2,73+2,73+2,73+2,79+2,57+2,53+2,53+2,53+2,57+2,51+2,39+2,39+2,39+2,51+2
409,245305,https://sofifa.com/player/245305/luis-roberto-...,L. Dálves,Luis Roberto Dálves,27,1992-02-29,188,83,Uruguay,Uruguay,81,81,0,0,"ST, CF",Right,1,3,3,High/Low,Normal,No,NaN,#Strength,ST,NaN,NaN,NaN,NaN,ST,9.0,80.0,82.0,60.0,71.0,34.0,75.0,NaN,NaN,NaN,...,65,32,81,58,77,79,28,27,24,8,11,20,19,20,80+2,80+2,80+2,73+2,76+2,76+2,76+2,73+2,71+2,71+2,71+2,70+2,63+2,63+2,63+2,70+2,53+2,51+2,51+2,51+2,53+2,51+2,51+2,51+2,51+2,51+2


In [ ]:
sub_data.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,...,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
48,175943,https://sofifa.com/player/175943/dries-mertens...,D. Mertens,Dries Mertens,32,1987-05-06,169,61,Belgium,Napoli,87,87,40000000,135000,"CF, ST",Right,3,4,4,High/Low,Normal,Yes,68000000.0,"#Dribbler, #Acrobat, #Clinical Finisher, #Comp...",CF,14.0,NaN,2013-07-01,2020.0,RF,14.0,88.0,83.0,80.0,91.0,35.0,53.0,NaN,NaN,NaN,...,59,36,87,83,79,81,25,40,40,7,10,8,10,4,78+3,78+3,78+3,86+3,85+3,85+3,85+3,86+3,85+3,85+3,85+3,85+3,78+3,78+3,78+3,85+3,65+3,60+3,60+3,60+3,65+3,60+3,46+3,46+3,46+3,60+3
53,193041,https://sofifa.com/player/193041/keylor-navas/...,K. Navas,Keylor Navas,32,1986-12-15,185,80,Costa Rica,Real Madrid,87,87,30500000,195000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,62500000.0,NaN,GK,1.0,NaN,2014-08-03,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,81.0,75.0,...,32,20,16,68,25,67,28,14,14,90,81,75,82,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,222492,https://sofifa.com/player/222492/leroy-sane/20...,L. Sané,Leroy Sané,23,1996-01-11,183,75,Germany,Manchester City,86,92,61000000,195000,LW,Left,2,3,4,High/Medium,Lean,Yes,125100000.0,"#Speedster, #Dribbler, #Acrobat",LW,19.0,NaN,2016-08-02,2021.0,LW,19.0,95.0,81.0,79.0,86.0,38.0,70.0,NaN,NaN,NaN,...,61,34,84,82,71,80,36,32,35,8,12,9,9,14,82+2,82+2,82+2,84+2,84+2,84+2,84+2,84+2,83+2,83+2,83+2,83+2,74+2,74+2,74+2,83+2,65+2,59+2,59+2,59+2,65+2,61+2,52+2,52+2,52+2,61+2
59,205600,https://sofifa.com/player/205600/samuel-umtiti...,S. Umtiti,Samuel Umtiti,25,1993-11-14,182,75,France,FC Barcelona,86,89,50000000,210000,CB,Left,3,3,2,Medium/High,Normal,Yes,106300000.0,"#Tackling , #Tactician",CB,23.0,NaN,2016-07-12,2023.0,LCB,5.0,69.0,63.0,69.0,69.0,86.0,82.0,NaN,NaN,NaN,...,82,87,51,58,61,82,87,88,85,15,10,14,12,15,68+3,68+3,68+3,66+3,67+3,67+3,67+3,66+3,67+3,67+3,67+3,67+3,72+3,72+3,72+3,67+3,77+3,80+3,80+3,80+3,77+3,79+3,84+3,84+3,84+3,79+3
62,197781,https://sofifa.com/player/197781/francisco-rom...,Isco,Francisco Román Alarcón Suárez,27,1992-04-21,176,79,Spain,Real Madrid,86,87,51000000,245000,"CAM, CM, LW",Right,3,3,4,Medium/Low,Normal,Yes,108400000.0,#Dribbler,CAM,22.0,NaN,2013-07-03,2022.0,LW,22.0,71.0,77.0,83.0,91.0,59.0,60.0,NaN,NaN,NaN,...,58,61,79,87,76,86,58,64,51,10,8,12,15,6,76+3,76+3,76+3,83+3,82+3,82+3,82+3,83+3,84+3,84+3,84+3,81+3,81+3,81+3,81+3,81+3,70+3,70+3,70+3,70+3,70+3,67+3,62+3,62+3,62+3,67+3


### Append all data into new Dataframe

In [ ]:
temp_data = nonSub_data.append(sub_data, ignore_index=True)

In [ ]:
new_data = temp_data.append(null_data, ignore_index=True)

In [ ]:
new_data["team_position"].isna().sum()

0

In [ ]:
new_data["team_position"].count()

15320

All data in team_position column already replaced and cleaned based on the data processing in above. After that, we can divide new_data dataframe to two groups, i.e. DataFrameTeamPositionGK_AllClub and DataFrameTeamPositionNonGK_AllClub

In [ ]:
new_data.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,...,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,NaN,NaN,NaN,...,48,40,94,94,75,96,33,37,26,6,11,15,14,8,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2018-07-10,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,NaN,NaN,NaN,...,63,29,95,82,85,95,28,32,24,7,11,15,14,11,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,2017-08-03,2022.0,LW,10.0,91.0,85.0,87.0,95.0,32.0,58.0,NaN,NaN,NaN,...,51,36,87,90,90,94,27,26,29,9,9,15,15,11,84+3,84+3,84+3,90+3,89+3,89+3,89+3,90+3,90+3,90+3,90+3,89+3,82+3,82+3,82+3,89+3,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,GK,13.0,NaN,2014-07-16,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,92.0,78.0,...,34,19,11,65,11,68,27,12,18,87,92,78,90,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",LW,7.0,NaN,2019-07-01,2024.0,LF,10.0,91.0,83.0,86.0,94.0,35.0,66.0,NaN,NaN,NaN,...,54,41,87,89,88,91,34,27,22,11,12,6,8,8,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,83+3,83+3,83+3,89+3,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


### Divided the new Dataframe into 2 part Dataframe
- DataFrameTeamPositionGK_AllClub 
- DataFrameTeamPositionNonGK_AllClub 

In [ ]:
gk_data = new_data.loc[new_data['team_position'] == "GK"]
nongk_data = new_data.loc[new_data['team_position'] != "GK"]

In [ ]:
gk_data['team_position'].count()

1630

In [ ]:
nongk_data['team_position'].count()

13690

## Create new CSV based on GK_data and NonGK_data

In [ ]:
gk_data.to_csv("DataFrameTeamPositionGK_AllClub.csv")
nongk_data.to_csv("DataFrameTeamPositionNonGK_AllClub.csv")